In [1]:
from __future__ import annotations

from datetime import  datetime
import ee
import geemap
from eelib import collections, eefuncs, bands

In [2]:
from dataclasses import dataclass, InitVar

@dataclass
class WillisCBAConfig:
    ASSET_ID: str = ""
    target_year: int = 2018
    doys: InitVar[tuple] = 135, 181, 182, 243, 244, 300
    band_prefix = {'spring': 'a_spri_b.*', 'summer': 'b_summ_b.*', 
                   'fall': 'c_fall_b.*'}
    def __post_init__(self, doys):
        def to_datetime(doys: tuple[str]):
            doys = sorted(doys)
            # julain days parse to YYYY MM dd
            def to_dt(doy: int) -> str:
                doy = datetime.strptime(f'{self.target_year}{doy}', '%Y%j')
                return doy.strftime("%Y-%m-%d")

            return tuple(map(to_dt, doys))
        
        dates = to_datetime(doys)
        
        self.seasons = {
            'spring': dates[:2],
            'summer': dates[2:4],
            'fall': dates[4:]
        }
        
        self.bands = [(b.name, b.value) for b in bands.WillistonCBA]


In [3]:
aoi = ee.FeatureCollection('users/ryangilberthamilton/BC/williston/williston_sub_a_2019')
dc = ee.ImageCollection('projects/fpca-336015/assets/williston-cba').\
    filterBounds(aoi)

img = dc.mosaic().select([str(_.name) for _ in bands.WillistonCBA], 
                         [str(_.value) for _ in bands.WillistonCBA])




In [11]:
spring = img.select('a_spri_b.*')
[str(_.name) for _ in bands.S2SR]
spring = spring.select(spring.bandNames(), [str(_.name) for _ in bands.S2SR])

spring.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

In [ ]:
TARGET_YEAR = '2018'

# set date ranges
doys = 135, 181, 182, 243, 244, 300 

def to_datetime(doys: tuple[str]):
    
    # julain days parse to YYYY MM dd
    def to_dt(doy: int) -> str:
        doy = datetime.strptime(f'{TARGET_YEAR}{doy}', '%Y%j')
        return doy.strftime("%Y-%m-%d")
    
    return tuple(map(to_dt, doys))
dates = to_datetime(doys)

spring, summer, fall = dates[:2], dates[2:4], dates[4:]
print(f'Spring:{spring}')
print(f'Summer:{summer}')
print(f'Fall:{fall}')
    


In [ ]:
viewport = collections.FeatureCollection.from_file(
    filename="../../000-data/intersection_64.shp",
    driver='ESRI Shapefile'
)

base = collections.S1Collection().\
    filterBounds(viewport).\
    filter('relativeOrbitNumber_start == 64').map(eefuncs.insert_groupid)

seasons = {
    'spring': {'collection': base.filterDate(*spring), 'date': spring},
    'summer': {'collection': base.filterDate(*summer),'date': summer},
    'fall': {'collection': base.filterDate(*fall),'date': fall}
}


In [ ]:
# set the mid date for each season and difference the days
for season, obj in seasons.items():
    dates = (ee.Date(_) for _ in obj.get('date'))
    mid_point = eefuncs.get_mid_point(dates)
    
    # update the collection inplace
    trg =  eefuncs.days_from_mid(
        collection=obj.get('collection'),
        midPoint = mid_point
    )
    
    trg = trg.map(lambda x: x.set('season', season))
    
    print(f"{season}: Days from Mid")
    print(trg.aggregate_array('dfm').distinct().sort().getInfo())
    print(trg.size().getInfo())

    obj['collection'] = trg

In [ ]:
# query the collections based on days

# Full coverage = dfms == [2, 16, 9]

spring_images = seasons.get('spring').get('collection').filter('dfm == 2')
summer_images = seasons.get('summer').get('collection').filter('dfm == 16')
fall_images = seasons.get('fall').get('collection').filter('dfm == 15')


In [ ]:
print(spring_images.aggregate_array('groupid').getInfo())
print(summer_images.aggregate_array('groupid').getInfo())
print(fall_images.aggregate_array('groupid').getInfo())

In [ ]:
col_list = [spring_images.toList(spring_images.size()),
            summer_images.toList(summer_images.size()),
            fall_images.toList(fall_images.size())]



In [ ]:
Map = geemap.Map()

Map

In [ ]:
vis = {'bands': ['VV'], 'min': -25, 'max': 10}

Map.addLayer(spring_images, vis, 'Spring')
Map.addLayer(summer_images, vis, 'Summer')
Map.addLayer(fall_images, vis, 'Fall')
Map.addLayer(viewport, {}, 'View Port')
Map.addLayerControl()
